# Werewolf Simulator
The following runs a simulation of the game Werewolf with multiple agents, it is built on top of and uses the Agentscope library, documentation can be found here https://doc.agentscope.io/, and the open source code that we extend on is here https://github.com/modelscope/agentscope/tree/main.
### Rules of Werewolf
- There are 4 roles, the Werewolves, Villagers, Witch, and Seer (Witch and Seer are on the Villager team)
- Each night:
    - The Werewolves discuss and vote on a player to eliminate
    - The Witch is told what player the wolves voted to eliminate, and is given the choice to use their potion of healing to reserruct the eliminated player, or use their poison to eliminate another player. Note that each power can only be used once in a game
    - The seer can pick any other player and find out what their role is (one player per night)
- After the events of the night, all surviving players discuss amongst themselves and vote on a player to eliminate
- Werewolves win if their numbers equal or exceed Villagers.
- Villagers win if all Werewolves are eliminated.

In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\ProgramData\anaconda3\python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

from typing import Optional, Union, Sequence, Any, List, Dict
from pydantic import BaseModel, Field
from functools import partial
import openai  
import faiss
import numpy as np
import json
import random 
import logging
import csv
from datetime import datetime
from zoneinfo import ZoneInfo
import sys
import time
from typing import Callable, Any

import numpy as np
from loguru import logger

from agentscope.parsers.json_object_parser import MarkdownJsonDictParser
from agentscope.parsers import ParserBase
from agentscope.message import Msg
from agentscope.msghub import msghub
from agentscope.agents import AgentBase
from agentscope.memory.temporary_memory import TemporaryMemory
from agentscope.pipelines.functional import sequentialpipeline
from agentscope.manager import ModelManager
import agentscope

from utils.werewolf_utils import (
    extract_name_and_id,
    n2s,
    set_parsers,
    generate_model_response
)

# Configure the Logger

In [3]:
# Default log configuration
LOG_LEVEL = logging.INFO
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
DATE_FORMAT = "%Y-%m-%d %H:%M:%S"

# Initial logging setup
logging.basicConfig(
    filename=None,
    level=LOG_LEVEL,
    format=LOG_FORMAT,
    datefmt=DATE_FORMAT
)

logger = logging.getLogger(__name__)

def change_log_file(new_filename: str):
    """
    Updates the logger handler to point to a different log file.
    """
    
    # Ensure the directory exists
    log_dir = os.path.dirname(new_filename)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir, exist_ok=True)  # Create directories if they don't exist

    # Remove all old handlers
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
        handler.close()

    # Create a new file handler
    new_handler = logging.FileHandler(new_filename)
    new_handler.setLevel(LOG_LEVEL)  # Reset level
    new_handler.setFormatter(logging.Formatter(LOG_FORMAT, datefmt=DATE_FORMAT))  # Reset formatter

    # Add new handler to logger
    logger.addHandler(new_handler)

def log_params(log_filepath: str, **kwargs) -> None:
    """
    Writes all parameters passed to the function into a JSON file.
    
    Args:
        log_filepath (str): Path where the JSON file will be written.
        **kwargs: Any number of keyword arguments representing parameter names and values.
    """
    with open(log_filepath, "w") as file:
        json.dump(kwargs, file, separators=(",", ":"))  # Compact JSON
        file.write("\n")  # Ensures a newline at the end

    print(f"Parameters written to {log_filepath}")

def log_vectorstore_retrieval(query: str, similar_documents: str):
    """Logs the most similar documents from vectorstore"""

    logger.info("===")
    logger.info(f"Query: {query}")
    logger.info(f"Retrieved similar documents: {similar_documents}")
    logger.info("===")

def log_message(msg: Msg):
    """Logs a message object""" 

    logger.info(msg.name + ": " + msg.content)


# TSV Logger

In [4]:
# Logging utilities
class StructuredLogger:
    """
    A logger that outputs to TSV files for structured logging of LLM interactions.
    """

    # Define all possible message types for documentation
    MESSAGE_TYPES = [
        "vectorstore_add",           # Adding a case to vectorstore
        "vectorstore_retrieve",      # Retrieving similar cases
        "vectorstore_summarize",     # Summarizing cached decisions
        "pro_argument_prompt",       # Input prompt for pro arguments
        "pro_argument_response",     # Response from pro argument LLM
        "con_argument_prompt",       # Input prompt for con arguments
        "con_argument_response",     # Response from con argument LLM
        "judge_prompt",              # Input prompt for judge
        "judge_response",            # Response from judge LLM
        "fast_prompt",               # Input prompt for fast LLM
        "fast_response",             # Fast LLM respnose
        "slow_prompt",               # Input prompt for slow LLM
        "slow_response",             # Slow LLM response
        
        # Werewolf-specific messages
        "phase",

        # Monopoly-specific messages
        "property_details",          # Property information being considered
        "player_stats",              # Player statistics at decision time
        "default_decision",          # Default player's decision
        "step_num",                  # Step number in game
        "game_init",                 # Game initialization config
        "player_summary"             # Players' state at decision time
    ]

    def __init__(self, tsv_filepath: str):
        """
        Initialize the logger with a base filepath.

        Args:
            tsv_filepath: Filepath for TSV log files (includes timestamp)
        """
        self.tsv_filepath = tsv_filepath

        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(self.tsv_filepath), exist_ok=True)

        # Initialize TSV file with headers if it doesn't exist
        if not os.path.exists(self.tsv_filepath):
            with open(self.tsv_filepath, "w", newline="", encoding="utf-8") as tsvfile:
                writer = csv.writer(tsvfile, delimiter="\t")
                writer.writerow(["timestamp", "message_type", "message"])

    def log(self, message_type: str, message_data: Union[str, Dict[str, Any], list[Any], BaseModel]) -> None:
        """
        Log a message to the TSV file.

        Args:
            message_type: Type of message (one of MESSAGE_TYPES)
            message_data: Data to be logged (will be converted to JSON if not already a string)
        """
        if message_type not in self.MESSAGE_TYPES:
            raise ValueError(
                f"Invalid message_type: {message_type}. Must be one of {self.MESSAGE_TYPES}"
            )

        # Convert message_data to string if it's not already
        if isinstance(message_data, BaseModel):
            message = json.dumps(message_data.model_dump(), ensure_ascii=False)
        elif isinstance(message_data, (dict, list)):
            message = json.dumps(message_data, ensure_ascii=False)
        else:
            message = str(message_data)

        timestamp = datetime.now().isoformat()

        # Append to TSV file
        with open(self.tsv_filepath, "a", newline="", encoding="utf-8") as tsvfile:
            writer = csv.writer(tsvfile, delimiter="\t", quotechar='`', quoting=csv.QUOTE_MINIMAL)
            writer.writerow([timestamp, message_type, message])

## Vector Store Implementations
Vector store classes with both uncompressed, and reflective functionalities

In [5]:
class ReflectiveVectorstoreMemory:
    """Reflective Vectorstore-based memory using FAISS and OpenAI embeddings."""
    
    def __init__(self, embedding_model: str = "text-embedding-ada-002"):
        """
        Initialize the vectorstore with FAISS and OpenAI embeddings.
        
        Args:
            embedding_model (str): The OpenAI embedding model to use.
        """
        self.embedding_model = embedding_model
        # Vector Store
        self.index = faiss.IndexFlatL2(1536)  # 1536 is the dimensionality of 'text-embedding-ada-002'
        self.messages = []  # To store actual messages (content)
        self.summaries = []  # To store summaries

    def _get_embedding(self, text: str) -> np.ndarray:
        """
        Generate an embedding for the given text using OpenAI.
        
        Args:
            text (str): The input text to embed.
        
        Returns:
            np.ndarray: The embedding vector as a NumPy array.
        """
        client = openai.OpenAI()

        response = retry_with_fallback(lambda: client.embeddings.create(
            input=text,
            model=self.embedding_model
        ))
        
        return np.array(response.data[0].embedding, dtype="float32")

    def add_message(self, message: Union[Msg, Sequence[Msg]]):
        """Add a message to the FAISS index."""
        if not isinstance(message, list):
            message = [message]
        for msg in message:
            self.messages.append(msg.name + ": " + msg.content)
        
    def summarize_cycle(self, survivors: list, secondary_model: str, cycle_type: str = "day", tsv_logger: Optional[StructuredLogger] = None):
        """
        Generate a summary of the conversation after a day/night cycle,
        and add it to the vector store.
        
        Args:
            survivors (list): List of current alive players.
            secondary_model (str): Name of secondary model.
            cycle_type (str): The type of cycle ("day" or "night").
        """
        client = openai.OpenAI()
        
        # Combine all messages since the last summary
        history = "\n".join(self.messages)

        # Use retry_with_fallback with a lambda function
        secondary_model_response = retry_with_fallback(
            lambda: generate_model_response(
                model=secondary_model,
                messages=[
                    {"role": "system", "content": "\n".join([ 
                        f"You are a strategic decision-maker reviewing past decisions in Werewolf.",
                        f"Survivors: {', '.join([survivor.name for survivor in survivors])}",
                        f"Summarize the decisions and reflect on the most important implications for future strategic decisions."
                    ])},
                    {"role": "user", "content": history},
                ]
            )
        )

        summary = secondary_model_response.text

        # Embed the summary and add it to the vector store
        embedding = self._get_embedding(summary)
        self.index.add(np.array([embedding]))
        self.summaries.append(summary)
        
        # add summarized vectorstore entry to tsv logger
        tsv_logger.log("vectorstore_summarize", {
            "messages": convert_messages_to_string(self.messages), 
            "summary": summary
        })

        # Clear messages for the next cycle
        self.messages.clear()

    def get_relevant_summaries_context(self, query: str, top_k: int = 1) -> str:
        """Retrieve the top-k most relevant summaries.
        
        Args:
            query (str): query to find similar summaries to.
            top_k (int): number of relevant summaries to retrieve.
        """
        if len(self.summaries) > 0:
            query_embedding = self._get_embedding(query)
            distances, indices = self.index.search(np.array([query_embedding]), top_k)
            results = [
                self.summaries[idx] for idx in indices[0] if idx < len(self.summaries)
            ]
            return "\n".join(list(set(results)))
        return ""
    
    def save_context(self, timestamp: str, game_num: int, player_role: str):
        """
        Save self.messages to a log file in the format:
        vectorstore/[timestamp]/[game_num]/[player_role].log

        Args:
            game_num (int): The game index.
            player_role (str): The role of the player.
        """

        # Construct the directory path
        dir_path = os.path.join("vectorstore", "fastslow", timestamp, str(game_num))

        # Ensure the directory exists
        os.makedirs(dir_path, exist_ok=True)

        # Construct the file path
        file_path = os.path.join(dir_path, f"{player_role}.log")

        # Save messages to the log file
        with open(file_path, "w", encoding="utf-8") as file:
            file.write("\n".join(self.summaries))

        print(f"Messages saved to {file_path}")

class VectorstoreMemory:
    """Vectorstore-based memory using FAISS and OpenAI embeddings."""
    
    def __init__(self, embedding_model: str = "text-embedding-ada-002"):
        """
        Initialize the vectorstore with FAISS and OpenAI embeddings.
        
        Args:
            embedding_model (str): The OpenAI embedding model to use.
        """
        self.embedding_model = embedding_model
        # Vector Store
        self.index = faiss.IndexFlatL2(1536)  # 1536 is the dimensionality of 'text-embedding-ada-002'
        self.messages = []  # To store actual messages (content)

    def _get_embedding(self, text: str) -> np.ndarray:
        """
        Generate an embedding for the given text using OpenAI.
        
        Args:
            text (str): The input text to embed.
        
        Returns:
            np.ndarray: The embedding vector as a NumPy array.
        """
        client = openai.OpenAI()
        response = retry_with_fallback(lambda: client.embeddings.create(
            input=text,
            model=self.embedding_model
        ))
        return np.array(response.data[0].embedding, dtype="float32")

    def add_message(self, message: Union[Msg, Sequence[Msg]], tsv_logger: Optional[StructuredLogger] = None):
        """Add a message to the FAISS index."""
        if not isinstance(message, list):
            message = [message]
        # add embedding of message to vector store and add log entry
        embeddings = []
        for msg in message:
            embeddings.append(self._get_embedding(msg.name + ": " + msg.content))
            tsv_logger.log("vectorstore_add", {
                "player": msg.name, 
                "content": msg.content
            })
        self.index.add(np.array(embeddings)) 
        self.messages.extend(message)

    def get_relevant_messages(self, query: str, top_k: int = 10) -> list:
        """Retrieve the top-k most relevant messages."""
        if not self.messages:
            return []
        query_embedding = self._get_embedding(query)
        distances, indices = self.index.search(np.array([query_embedding]), top_k)

        results = [
            self.messages[idx] for idx in indices[0] if idx < len(self.messages)
        ]
        return results

    def save_context(self, timestamp: str, game_num: int, player_role: str):
        """
        Save self.messages to a log file in the format:
        vectorstore/[timestamp]/[game_num]/[player_role].log

        Args:
            game_num (int): The game index.
            player_role (str): The role of the player.
        """
        # Construct the directory path
        dir_path = os.path.join("vectorstore", "fastslow", timestamp, str(game_num))

        # Ensure the directory exists
        os.makedirs(dir_path, exist_ok=True)

        # Construct the file path
        file_path = os.path.join(dir_path, f"{player_role}.log")

        # Save messages to the log file
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(convert_messages_to_string(self.messages))

        print(f"Messages saved to {file_path}")

        

## Custom Agents (edit here)
Below are implementations of a custom Agent utilitizing the Slow Mind Fast Mind Architecture with a reflective vector store, and an agent that generates responses based off a normal vector store search.

In [6]:
class SlowFastMindAgent(AgentBase):
    """An agent that generates response using the Fast Mind Slow Mind Architecture.
    
    Includes a FAISS vectorstore for memory
    """

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        fast_mind_model_config_name: str,
        slow_mind_model_config_name: str,
        reflect_before_vectorstore: bool = False,
        similarity_top_k: int = 1,
        openai_api_key: str = "",
        tsv_logger: Optional[StructuredLogger] = None
    ) -> None:
        """Initialize the custom agent.

        Arguments:
            name (`str`):
                The name of the agent.
            sys_prompt (`Optional[str]`, defaults to `None`):
                The system prompt of the agent, which can be passed by args
                or hard-coded in the agent.
            fast_mind_model_config_name (`str`, defaults to None):
                The name of the model config, which is used to load fast mind 
                model from configuration.
            slow_mind_model_config_name (`str`, defaults to None):
                The name of the model config, which is used to load slow mind 
                model from configuration.
            max_retries (`Optional[int]`, defaults to `None`):
                The maximum number of retries when failed to parse the model
                output.      
            similarity_top_k (`int`, defaults to `None`):
                The top k most similar items in the vectorstore that will be
                used in the context to the model
            openai_api_key (`str`)
        """
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
        )
        self.parser = None
        self.reflect_before_vectorstore = reflect_before_vectorstore
        self.similarity_top_k = similarity_top_k
        self.fast_mind_model_config_name = fast_mind_model_config_name 
        self.slow_mind_model_config_name = slow_mind_model_config_name
        self.tsv_logger=tsv_logger

        openai.api_key = openai_api_key

        # Load the two models from Agentscope's ModelManager
        model_manager = ModelManager.get_instance()
        self.fast_mind_model = model_manager.get_model_by_config_name(fast_mind_model_config_name)
        self.slow_mind_model = model_manager.get_model_by_config_name(slow_mind_model_config_name)

        if self.reflect_before_vectorstore:
            self.memory = ReflectiveVectorstoreMemory()
        else:
            self.memory = VectorstoreMemory()

    def set_parser(self, parser: ParserBase) -> None:
        """Set response parser, which will provide 1) format instruction; 2)
        response parsing; 3) filtering fields when returning message, storing
        message in memory. So developers only need to change the
        parser, and the agent will work as expected.
        """
        self.parser = parser

    def _build_prompt(self, x: Optional[Msg]):
        """Helper to build a prompt for either fast or slow mind."""
        
        query = Queries.werewolf_discussion_query
        if self.reflect_before_vectorstore:
            summary_context = self.memory.get_relevant_summaries_context(query=query, top_k=self.similarity_top_k)

            # Log file entry
            log_vectorstore_retrieval(query, summary_context)
            
            # Log vector store retrieval results to structured TSV
            if self.tsv_logger:
                serializable_results = [
                    {"content": doc, "query": query}
                    for doc in summary_context
                ]
                self.tsv_logger.log("vectorstore_retrieve", serializable_results)

            prompt = self.fast_mind_model.format(
                Msg("system", self.sys_prompt, role="system"),
                Msg("system", f"Summary of relevant past conversations: {summary_context}", role="system"),
                Msg("system", self.parser.format_instruction, role="system"),
            )
        else:
            relevant_msgs = self.memory.get_relevant_messages(query, self.similarity_top_k)
            prompt = self.fast_mind_model.format(
                Msg("system", self.sys_prompt, role="system"),
                relevant_msgs and Msg("system", f"Relevant messages from memory: {relevant_msgs}", "system"),
                x,
                Msg("system", self.parser.format_instruction, role="system"),
            )
            log_vectorstore_retrieval(query, convert_messages_to_string(relevant_msgs))
            
            # Log vector store retrieval results to structured TSV
            if self.tsv_logger:
                serializable_results = [
                    {"content": doc.name + ": " + doc.content, "query": query}
                    for doc in relevant_msgs
                ]
                self.tsv_logger.log("vectorstore_retrieve", serializable_results)
        return prompt

    def reply(self, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """Reply function of the agent.
        Processes the input data, generates a prompt using the current
        dialogue memory and system prompt, and invokes the language
        model to produce a response. The response is then formatted
        and added to the dialogue memory.

        Args:
            x (`Optional[Union[Msg, Sequence[Msg]]]`, defaults to `None`):
                The input message(s) to the agent, which also can be omitted if
                the agent doesn't need any input.

        Returns:
            `Msg`: The output message generated by the agent.

        Raises:
            `json.decoder.JSONDecodeError`:
                If the response from the language model is not valid JSON,
                it defaults to treating the response as plain text.
        """     
        model_prompt = self._build_prompt(x)

        raw_response= retry_with_fallback(
            lambda: generate_model_response(self.fast_mind_model_config_name, model_prompt)
        )

        self.speak(raw_response.text)

        # Parse the raw response
        parsed_response = self.parser.parse(raw_response).parsed

        vectorstore_entry = Msg(
            self.name,
            content=json.dumps(parsed_response),
            role="assistant",
        )

        # Save the response to memory
        if self.reflect_before_vectorstore:
            self.memory.add_message(message=vectorstore_entry)
        else:
            self.memory.add_message(message=vectorstore_entry, tsv_logger=self.tsv_logger)

        msg = Msg(
            self.name,
            content=self.parser.to_content(parsed_response),
            role="assistant",
            metadata=self.parser.to_metadata(parsed_response),
        )

        return msg
    
    def fast_slow_mind(self, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """Fast Mind Slow Mind decision making function of the agent.

        Args:
            x (`Optional[Union[Msg, Sequence[Msg]]]`, defaults to `None`):
                The input message(s) to the agent, which also can be omitted if
                the agent doesn't need any input.

        Returns:
            `Msg`: The output message generated by the agent.

        Raises:
            `json.decoder.JSONDecodeError`:
                If the response from the language model is not valid JSON,
                it defaults to treating the response as plain text.
        """     
        model_prompt = self._build_prompt(x)
        
        # tsv log fast prompt
        self.tsv_logger.log("fast_prompt", model_prompt[0]["content"])
        
        fast_resp = retry_with_fallback(
            lambda: generate_model_response(self.fast_mind_model_config_name, model_prompt)
        )

        # Parse the raw response
        parsed_response = self.parser.parse(fast_resp).parsed

        # Obtain 'uncertainty_score'
        try:
            data = json.loads(fast_resp.text.replace("```json", "").replace("```", "").strip())
            uncertainty = float(data.get("uncertainty_score", 0.0))
            thought = data.get("thought")
            vote = data.get("vote")
        except:
            uncertainty = 1.0
            thought = ""
            vote = ""
        
        logger.info("===")
        logger.info(f"Fast Response: {json.dumps(parsed_response)}")
        logger.info(f"Fast Uncertainty: {uncertainty}")
        logger.info("===")
        
        # tsv log fast response
        self.tsv_logger.log("fast_response", {
            "thought": thought,
            "vote": vote,
            "uncertainty": uncertainty
        })

        if uncertainty >= 0.6:
            # tsv log slow prompt
            self.tsv_logger.log("slow_prompt", model_prompt[0]["content"])
            raw_response = retry_with_fallback(
                lambda: generate_model_response(self.slow_mind_model_config_name, model_prompt)
            )

            parsed_response = self.parser.parse(raw_response).parsed

            logger.info("===")
            logger.info(f"Slow Response: {json.dumps(parsed_response)}")
            logger.info("===")
            
            # tsv log slow response
            self.tsv_logger.log("slow_response", parsed_response)
        else:
            raw_response = fast_resp

        self.speak(raw_response.text)

        vectorstore_entry = Msg(
            self.name,
            content=json.dumps(parsed_response),
            role="assistant",
        )

        # Save the response to memory
        if self.reflect_before_vectorstore:
            self.memory.add_message(message=vectorstore_entry)
        else:
            self.memory.add_message(message=vectorstore_entry, tsv_logger=self.tsv_logger)

        msg = Msg(
            self.name,
            content=self.parser.to_content(parsed_response),
            role="assistant",
            metadata=self.parser.to_metadata(parsed_response),
        )

        return msg
    
    def observe(self, x: Union[Msg, Sequence[Msg]], role: str = None) -> None:
        """
        Record incoming messages in memory without generating a reply.

        Args:
            x (Union[Msg, Sequence[Msg]]): The message(s) to be stored in memory for future context.
            role (`str`, optional): Role of the observer
        """
        if role == "seer" and x is not None:
            if self.reflect_before_vectorstore:
                self.memory.add_message(message=vectorstore_entry)
            else:
                self.memory.add_message(message=vectorstore_entry, tsv_logger=self.tsv_logger)
            
    def summarize_cycle(self, survivors: list, slow_mind_model: str, cycle_type: str = "day"):
        """
        Generate a summary at the end of a day or night cycle.
        
        Args:
            survivors: (list): List of current survivors.
            slow_mind_model (str): Name of the slow mind model.
            cycle_type (str): The type of cycle ("day" or "night").
        """
        if self.reflect_before_vectorstore:
            self.memory.summarize_cycle(survivors, slow_mind_model, cycle_type=cycle_type, tsv_logger=self.tsv_logger)

    def save_context(self, timestamp: str, game_num: int, player_role: str="werewolf"):
        """
        Save self.messages to a log file in the format:
        vectorstore/[timestamp]/[game_num]/[player_role].log

        Args:
            game_num (int): The game index.
            player_role (str): The role of the player.
        """
        
        self.memory.save_context(timestamp, game_num, player_role)
        
class NormalAgent(AgentBase):
    """Agent with normal uncompressed VectorStore,
    no other underlying decision making architecture.
    """

    def __init__(
        self,
        name: str,
        sys_prompt: str,
        model_config_name: str,
        max_retries: Optional[int] = 3,
        similarity_top_k: int = None,
        openai_api_key: str = "",
        tsv_logger: Optional[StructuredLogger] = None
    ) -> None:
        """Initialize the dict dialog agent.

        Arguments:
            name (`str`):
                The name of the agent.
            sys_prompt (`Optional[str]`, defaults to `None`):
                The system prompt of the agent, which can be passed by args
                or hard-coded in the agent.
            model_config_name (`str`, defaults to None):
                The name of the model config, which is used to load model from
                configuration.
            max_retries (`Optional[int]`, defaults to `None`):
                The maximum number of retries when failed to parse the model
                output.
        """  # noqa
        super().__init__(
            name=name,
            sys_prompt=sys_prompt,
            model_config_name=model_config_name,
        )

        self.parser = None
        self.max_retries = max_retries
        openai.api_key = openai_api_key  # Set OpenAI API key
        self.memory = VectorstoreMemory()  # Use FAISS vector store for memory
        self.similarity_top_k = similarity_top_k
        self.model_config_name = model_config_name
        self.tsv_logger=tsv_logger

    def set_parser(self, parser: ParserBase) -> None:
        """Set response parser, which will provide 1) format instruction; 2)
        response parsing; 3) filtering fields when returning message, storing
        message in memory. So developers only need to change the
        parser, and the agent will work as expected.
        """
        self.parser = parser

    def reply(self, x: Optional[Union[Msg, Sequence[Msg]]] = None) -> Msg:
        """Reply function of the agent.
        Processes the input data, generates a prompt using the current
        dialogue memory and system prompt, and invokes the language
        model to produce a response. The response is then formatted
        and added to the dialogue memory.

        Args:
            x (`Optional[Union[Msg, Sequence[Msg]]]`, defaults to `None`):
                The input message(s) to the agent, which also can be omitted if
                the agent doesn't need any input.

        Returns:
            `Msg`: The output message generated by the agent.

        Raises:
            `json.decoder.JSONDecodeError`:
                If the response from the language model is not valid JSON,
                it defaults to treating the response as plain text.
        """     
        query = Queries.non_werewolf_discussion_query

        relevant_msgs = self.memory.get_relevant_messages(query=query, top_k=self.similarity_top_k)

        log_vectorstore_retrieval(query, convert_messages_to_string(relevant_msgs))
        
        # Log vector store retrieval results to structured TSV
        if self.tsv_logger:
            serializable_results = [
                {"content": doc.name + ": " + doc.content, "query": query}
                for doc in relevant_msgs
            ]
            self.tsv_logger.log("vectorstore_retrieve", serializable_results)

        # prepare prompt with retrieved messages similar to input message
        prompt = self.model.format(
            Msg("system", self.sys_prompt, role="system"),
            relevant_msgs and Msg("system", f"Relevant messages from memory: {relevant_msgs}", "system"),
            x,
            Msg("system", self.parser.format_instruction, "system"),
        )

        # Call llm
        raw_response = retry_with_fallback(
            lambda: generate_model_response(self.model_config_name, prompt)
        )

        self.speak(raw_response.text)

        # Parsing the raw response
        parsed_response = self.parser.parse(raw_response).parsed

        vectorstore_entry = Msg(
            self.name,
            content=json.dumps(parsed_response),
            role="assistant",
        )

        # Save the response to memory
        self.memory.add_message(message=vectorstore_entry, tsv_logger=self.tsv_logger)

        msg = Msg(
            self.name,
            content=self.parser.to_content(parsed_response),
            role="assistant",
            metadata=self.parser.to_metadata(parsed_response),
        )

        return msg
    
    def observe(self, x: Union[Msg, Sequence[Msg]], role: str = None) -> None:
        """
        Record incoming messages in memory without generating a reply.

        Args:
            x (Union[Msg, Sequence[Msg]]): The message(s) to be stored in memory for future context.
            role (`str`, optional): Role of the observer
        """
        if role == "seer" and x is not None:
            self.memory.add_message(message=x, tsv_logger=self.tsv_logger)

    def save_context(self, timestamp: str, game_num: int, player_role: str="werewolf"):
        """
        Save self.messages to a log file in the format:
        vectorstore/[timestamp]/[game_num]/[player_role].log

        Args:
            game_num (int): The game index.
            player_role (str): The role of the player.
        """
        
        self.memory.save_context(timestamp, game_num, player_role)
            
            
# Register custom classes
agentscope.agents.SlowFastMindAgent = SlowFastMindAgent
agentscope.agents.NormalAgent = NormalAgent

## Prompts and Parser (edit minimally)
Below are the prompts that control and run the Werwolf game. They are implemented using the built in AgentScope MarkdownJsonDictParser which allows us to generate responses in a dictionary format that is compatible with our game. Currently structured to obtain both the private thoughts of an Agent which remain to themselves and showcase their reasoning, and the words that they speak to other agents.

Since these prompts control the game logic itself, we should try to limit the prompt edits here to a minimum, but we can make edits to the parser to control the reasoning process a particular role goes through before making a decision, eg. instead of a thought field, we ask it to make ask it to provide a reason to vote for a particular agent, and a reason to not vote for a particular agent, then ask it to make a final decision based off those fields. Fields other than "thought" however should not be touched as they play a critical role to the control flow of the game.

More details on the AgentScope parsers can be found here https://doc.agentscope.io/build_tutorial/structured_output.html

In [7]:
class Prompts:
    """Prompts for werewolf game"""

    rules = """
        Player roles: In werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.
        - Werewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.
        - Villagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.
        - Seer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.
        - Witch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).

        Game Rule: The game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.
        1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).
        2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.

        Victory Condition: For werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers. For villagers, they win if they identify and eliminate all of the werewolves in the group.

        Constraints:
        1. Your response should be in the first person.
        2. This is a conversational game. You should response only based on the conversation history and your strategy.
    """

    two_phases = """
    """

    to_wolves = (
        "{}, if you are the only werewolf, eliminate a player. Otherwise, "
        "discuss with your teammates and reach an agreement."
    )

    wolves_discuss_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
            "finish_discussion": "whether the discussion reached an agreement or not (true/false)",
        },
        required_keys=["thought", "speak", "finish_discussion"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["finish_discussion"],
    )

    to_wolves_vote = "Which player do you vote to kill?"

    # Adding uncertainty score for Fast Mind Slow Mind Architecture
    wolves_vote_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "vote": "player_name",
            "uncertainty_score": "score between 0-1 on how uncertain you are about your vote, 0 if your completely certain, 1 if your completely uncertain",
        },
        required_keys=["thought", "vote", "uncertainty_score"],
        keys_to_memory="vote",
        keys_to_content="vote",
    )

    to_wolves_res = "The player with the most votes is {}."

    to_witch_resurrect = (
        "{witch_name}, you're the witch. Tonight {dead_name} is eliminated. "
        "Would you like to resurrect {dead_name}?"
    )

    to_witch_resurrect_no = "The witch has chosen not to resurrect the player."
    to_witch_resurrect_yes = "The witch has chosen to resurrect the player."

    witch_resurrect_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "whether to resurrect the player and the reason",
            "resurrect": "whether to resurrect the player or not (true/false)",
        },
        required_keys=["thought", "speak", "resurrect"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["resurrect"],
    )

    to_witch_poison = (
        "Would you like to eliminate one player? If yes, "
        "specify the player_name."
    )

    witch_poison_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
            "eliminate": "whether to eliminate a player or not (true/false)",
        },
        required_keys=["thought", "speak", "eliminate"],
        keys_to_memory="speak",
        keys_to_content="speak",
        keys_to_metadata=["eliminate"],
    )

    to_seer = (
        "{}, you're the seer. Which player in {} would you like to check "
        "tonight?"
    )

    seer_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "player_name",
        },
        required_keys=["thought", "speak"],
        keys_to_memory="speak",
        keys_to_content="speak",
    )

    to_seer_result = "Okay, the role of {} is a {}."

    to_all_danger = (
        "The day is coming, all the players open your eyes. Last night, "
        "the following player(s) has been eliminated: {}."
    )

    to_all_peace = (
        "The day is coming, all the players open your eyes. Last night is "
        "peaceful, no player is eliminated."
    )

    to_all_discuss = (
        "Now the alive players are {}. Given the game rules and your role, "
        "based on the situation and the information you gain, to vote a "
        "player eliminated among alive players and to win the game, what do "
        "you want to say to others? You can decide whether to reveal your "
        "role."
    )

    survivors_discuss_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "speak": "what you speak",
        },
        required_keys=["thought", "speak"],
        keys_to_memory="speak",
        keys_to_content="speak",
    )

    # Adding uncertainty score for Fast Mind Slow Mind Architecture
    survivors_vote_parser = MarkdownJsonDictParser(
        content_hint={
            "thought": "what you thought",
            "vote": "player_name",
            "uncertainty_score": "score between 0-1 on how uncertain you are about your vote, 0 if your completely certain, 1 if your completely uncertain",

        },
        required_keys=["thought", "vote", "uncertainty_score"],
        keys_to_memory="vote",
        keys_to_content="vote",
    )

    to_all_vote = (
        "Given the game rules and your role, based on the situation and the"
        " information you gain, to win the game, it's time to vote one player"
        " eliminated among the alive players. Which player do you vote to "
        "kill?"
    )

    to_all_res = "{} has been voted out."

    to_all_wolf_win = (
        "The werewolves have prevailed and taken over the village. Better "
        "luck next time!"
    )

    to_all_village_win = (
        "The game is over. The werewolves have been defeated, and the village "
        "is safe once again!"
    )

    to_all_continue = "The game goes on."

    
# Moderator message function
HostMsg = partial(Msg, name="Moderator", role="assistant", echo=True)

# Vectorstore Queries

In [8]:
class Queries:
    """Queries for the vectorstore based on certain points in game"""

    # Retrieves discussions about suspicions of non-werewolves to help identify potential enemies.
    werewolf_discussion_query = "Discussions related to villagers, seer, witch."

    # Retrieves discussions focused on identifying werewolves.
    non_werewolf_discussion_query = "Discussions related to werewolves."

## Game Initialization (edit here)
To initialize the agents, you must define their parameters and settings in the config objects that are passed in for initialization. There is a model config, which defines the base foundational model being used, and an agent config, which defines each of the agents being used in the game, including which model their using, their name, and what type of Agent they are (based off the agent classes we defined earlier). 

Pay particular attention to the system prompt, this is what defines the rules of the game to the agent and gives them the role and what their responsibilities are, we could perhaps do some prompt engineering with that.

Also we can play around with the settings of the game, eg. max rounds, how many werewolves we have, etc. Just make sure to update the roles, witch, seer objects below accordingly.

In [9]:
# Load API keys from .env file
load_dotenv()

# Define the API keys
openai_key = os.getenv("OPENAI_API_KEY")

## Utility Functions (don't edit)
Functions to check and update game state throughout

In [10]:
def retry_with_fallback(func: Callable[..., Any], *args: Any, **kwargs: Any) -> Any:
    """
    Tries to execute the given function. If it fails, waits 5 seconds and retries once.
    If it fails again, exits the script.

    Args:
        func (Callable[..., Any]): The function to execute.
        *args (Any): Positional arguments to pass to the function.
        **kwargs (Any): Keyword arguments to pass to the function.

    Returns:
        Any: The result of the function if successful.
    """
    try:
        return func(*args, **kwargs)  # Try executing the function
    except Exception as e:
        print(f"Error: {e}. Retrying in 5 seconds...")
        time.sleep(5)  # Wait 5 seconds before retrying

        try:
            return func(*args, **kwargs)  # Try again
        except Exception as e:
            print(f"Error: {e}. Failed again. Stopping execution.")
            sys.exit(1)  # Exit the script if it fails twice

def load_json_response(response: str) -> dict:
    """Converts a json response to a python dictionary"""
    return json.loads(response.replace("```json", "").replace("```", "").strip())

def convert_messages_to_string(relevant_messages: list) -> str:
    """Takes a list of messages, and converts them to a string separated by new line"""
    return '\n'.join([f"{msg.name}: {msg.content}" for msg in relevant_messages])

def majority_vote(votes: list) -> Any:
    """Given a list of votes, return the name with the highest frequency."""
    votes_valid = [v for v in votes if v != "Abstain"]
    if not votes_valid:
        return "No Votes"
    unique_vals, counts = np.unique(votes_valid, return_counts=True)
    return unique_vals[np.argmax(counts)]

def update_alive_players(game_state: dict, survivors: list, wolves: list, dead_names):
    """
    Removes 'dead_names' from 'game_state["survivors"]', updates game_state["dead"],
    and returns updated survivors and wolves lists.
    """
    if not isinstance(dead_names, list):
        dead_names = [dead_names]
    for d in dead_names:
        if d in game_state["survivors"]:
            game_state["survivors"].remove(d)
        if d not in game_state["dead"]:
            game_state["dead"].append(d)

    # Rebuild survivors / wolves lists
    new_survivors = [s for s in survivors if s.name in game_state["survivors"]]
    new_wolves = [w for w in wolves if w.name in game_state["survivors"]]
    return new_survivors, new_wolves

def check_winning(game_state: dict, survivors: list, wolves: list, host: str) -> bool:
    """
    If #werewolves * 2 >= #survivors => werewolves instantly win.
    If all werewolves are dead => villagers instantly win.
    Otherwise => game continues.
    """
    if len(wolves) * 2 >= len(survivors):
        msg = Msg(host, Prompts.to_all_wolf_win, role="assistant")
        game_state["endgame"] = True
        game_state["winner"] = "werewolves"
        log_message(msg)
        return True
    if survivors and not wolves:
        msg = Msg(host, Prompts.to_all_village_win, role="assistant")
        game_state["endgame"] = True
        game_state["winner"] = "villagers"
        logger.info(f"Moderator: {Prompts.to_all_village_win}")
        log_message(msg)
        return True
    return False

def generate_log_filepath(basepath: str, 
                          reflect_before_vectorstore: bool, 
                          random_seed: int, 
                          timestamp: str, 
                          game_num: int) -> str:
    """Generate log file path with EST timestamp."""
    return f"{basepath}/{'reflect' if reflect_before_vectorstore else 'noreflect'}/{random_seed}/{timestamp}/fastslow_raw_game{game_num}.log"

def save_results(win_rate_filepath: str, row: dict):
    """
    Append a row to a CSV file. If doesn't exist, create with headers.

    row = {
      "raw_log_filepath": <str>,
      "custom_agent_won": <bool>,
    }
    """
    file_exists = os.path.isfile(win_rate_filepath)
    with open(win_rate_filepath, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["raw_log_filepath", "custom_agent_won"])
        if not file_exists:
            w.writeheader()
        w.writerow(row)
    print(f"Row added to {win_rate_filepath}: {row}")

def custom_agent_won(game_state: dict, alive_agents: list, wolf_agents: list):
    """Records whether custom agent won (even if game didn't complete)"""

    # If game is completed, check if werewolves have won
    if game_state["endgame"]:
        if game_state["winner"] == "werewolves":
            return True 
        return False
    
    # If game not completed, then custom agent wins because it is still alive and eliminated another player
    return True

def get_available_targets(survivors: list, wolves: list):
    """Returns list of currently alive non-werewolves"""
    return [survivor.name for survivor in survivors if survivor not in wolves]

## Running a Game 
Following is a function that game through the various Night and Day phases, taking different actions for each agent based on their roles. Multi agent functionality and communication is facilitated through the AgentScope Pipeline and MsgHub, more detailed documentation found here https://doc.agentscope.io/build_api/agentscope.pipelines.pipeline.html#module-agentscope.pipelines.pipeline and https://doc.agentscope.io/build_api/agentscope.msghub.html#module-agentscope.msghub

In [11]:
def run_game(
    slow_mind_model: str,
    max_days_per_game: int,
    reflect_before_vectorstore: bool,
    max_werewolf_discussion_round: int,
    wolves: list,
    seer,
    witch,
    roles,
    survivors,
    game_state,
    tsv_logger: Optional[StructuredLogger] = None
):
    """
    Runs a single game, storing logs in game_log. This example merges both
    day/night flow, werewolf talk, witch usage, seer usage, etc. Then
    saves logs and results.
    """
    for day_i in range(1, max_days_per_game + 1):
        
        tsv_logger.log("phase", "Night Time")

        # 1) Night Phase: Werewolves discussion
        hint = HostMsg(content=Prompts.to_wolves.format(n2s(wolves)))
        log_message(hint)
        logger.info(f"Moderator: {hint.content}")
        with msghub(wolves, announcement=hint) as hub:
            set_parsers(wolves, Prompts.wolves_discuss_parser)
            for r in range(max_werewolf_discussion_round):
                x = retry_with_fallback(lambda: sequentialpipeline(wolves))
                logger.info(f"Werewolves discussion: {x.content}")
                if x.metadata.get("finish_discussion", False):
                    break
            # Then vote
            set_parsers(wolves, Prompts.wolves_vote_parser)
            hint_vote = HostMsg(content=Prompts.to_wolves_vote)
            log_message(hint_vote)
            logger.info(f"Moderator: {hint_vote.content}")
            votes = [extract_name_and_id(wolf.fast_slow_mind(hint_vote).content)[0] for wolf in wolves]
            
            voted_out = majority_vote(votes)
            dead_player = [voted_out]

            voted_out_hint = HostMsg(content=Prompts.to_wolves_res.format(voted_out))
            log_message(voted_out_hint)
            hub.broadcast(voted_out_hint)

        # 2) Witch Decision Night   
        healing_used_tonight = False
        if witch in survivors:
            if not game_state["witch_healing_used"]:
                hint = HostMsg(
                    content=Prompts.to_witch_resurrect.format_map(
                        {
                            "witch_name": witch.name,
                            "dead_name": dead_player[0],
                        },
                    ),
                )
                log_message(hint)
                set_parsers(witch, Prompts.witch_resurrect_parser)
                
                # Capture the witch's resurrection response and log it
                resurrection_response = retry_with_fallback(lambda: witch(hint))
                logger.info(f"Witch resurrection response: {resurrection_response.content}")

                if resurrection_response.metadata.get("resurrect", False):
                    healing_used_tonight = True
                    dead_player.pop()
                    game_state["witch_healing_used"] = True
                    HostMsg(content=Prompts.to_witch_resurrect_yes)
                    log_message(HostMsg(content=Prompts.to_witch_resurrect_yes))
                else:
                    HostMsg(content=Prompts.to_witch_resurrect_no)
                    log_message(HostMsg(content=Prompts.to_witch_resurrect_no))

            if not game_state["witch_poison_used"] and not healing_used_tonight:
                set_parsers(witch, Prompts.witch_poison_parser)
                
                # Capture the witch's poison response and log it
                poison_response = retry_with_fallback(lambda: witch(HostMsg(content=Prompts.to_witch_poison)))
                logger.info(f"Witch Poison Response: {poison_response.content}")

                if poison_response.metadata.get("eliminate", False):
                    target_player = extract_name_and_id(poison_response.content)[0]
                    dead_player.append(target_player)
                    game_state["witch_poison_used"] = True
                    logger.info(f"The witch has chosen to poison {target_player}.")
                else:
                    logger.info("The witch has chosen not to poison a player.")

        # 3) Seer checks a role
        if seer in survivors:
            seer_hint = HostMsg(content=Prompts.to_seer.format(seer.name, n2s(survivors)))
            log_message(seer_hint)
            set_parsers(seer, Prompts.seer_parser)
            x = seer.reply(seer_hint)
            logger.info(f"Seer response: {x.content}")
            pl, idx = extract_name_and_id(x.content)
            role_name = roles[idx] if idx < len(roles) else "villager"
            # Let seer quietly observe the result
            role_msg = HostMsg(content=Prompts.to_seer_result.format(pl, role_name))
            logger.info(f"Moderator: {role_msg.content}")
            seer.observe(role_msg, "seer")

        # 4) Update survivors after night
        survivors, wolves = update_alive_players(game_state, survivors, wolves, dead_player)
        if check_winning(game_state, survivors, wolves, "Moderator"):
            break

        # If reflecting, do a night reflection
        if reflect_before_vectorstore:
            for w in wolves:
                w.summarize_cycle(survivors, slow_mind_model, "night", tsv_logger)

        tsv_logger.log("phase", "Day Time")
                
        # 5) Daytime discussion
        content = (
            Prompts.to_all_danger.format(n2s(dead_player))
            if dead_player
            else Prompts.to_all_peace
        )
        discuss_hints = [
            HostMsg(content=content),
            HostMsg(content=Prompts.to_all_discuss.format(n2s(survivors))),
        ]
        with msghub(survivors, announcement=discuss_hints) as hub:
            set_parsers(survivors, Prompts.survivors_discuss_parser)
            discussion_out = retry_with_fallback(lambda: sequentialpipeline(survivors))
            logger.info(f"Survivors Discussion: {discussion_out.content}")

            # Daytime vote
            set_parsers(survivors, Prompts.survivors_vote_parser)
            day_vote_hint = HostMsg(content=Prompts.to_all_vote)
            log_message(day_vote_hint)
            # use fast mind slow mind for werewolf vote
            votes_day = [
                extract_name_and_id(wolf.fast_slow_mind(day_vote_hint).content)[0] for wolf in wolves
            ]

            votes_day.extend([
                extract_name_and_id(_.reply(day_vote_hint).content)[0] for _ in survivors if _ not in wolves
            ])
            day_result = majority_vote(votes_day)
            day_result_out = HostMsg(content=Prompts.to_all_res.format(day_result))
            hub.broadcast(day_result_out)
            log_message(day_result_out)

            survivors, wolves = update_alive_players(game_state, survivors, wolves, day_result)
            if check_winning(game_state, survivors, wolves, "Moderator"):
                break

            # If reflecting, do a day reflection
            if reflect_before_vectorstore:
                for w in wolves:
                    w.summarize_cycle(survivors, slow_mind_model, "day", tsv_logger)

            hub.broadcast(HostMsg(content=Prompts.to_all_continue))

    return custom_agent_won(
        game_state,
        survivors,
        wolves
    )


## Main Entry: run_experiment()

In [12]:
def run_experiment(
    open_ai_key: str,
    basefilepath: str,
    max_days_per_game: int = 3,
    num_games: int = 2,
    reflect_before_vectorstore: bool = False,
    fast_mind_model: str = "gpt-4o-mini",
    slow_mind_model: str = "gpt-4o",
    max_werewolf_discussion_round: int = 3,
    similarity_top_k: int = 1,
    random_seed: int = 42
) -> None:
    """
    Initialize and run multiple games of the Werewolf simulation.

    Arguments:
        open_ai_key (`str`):
            The OpenAI API key used for model calls.
        basefilepath (`str`):
            Base path for logs, e.g., "logs/game_raw".
        max_days_per_game (`int`, defaults to `3`):
            Maximum number of day/night cycles per game.
        num_games (`int`, defaults to `2`):
            Number of games to simulate.
        reflect_before_vectorstore (`bool`, defaults to `False`):
            Whether to use reflective memory before vectorstore retrieval.
        fast_mind_model (`str`, defaults to `"gpt-4o-mini"`):
            Name of the model used for fast processing.
        slow_mind_model (`str`, defaults to `"gpt-4o"`):
            Name of the model used for complex reasoning.
        max_werewolf_discussion_round (`int`, defaults to `3`):
            Maximum number of discussion rounds for werewolves.
        similarity_top_k (`int`, defaults to `1`):
            Number of messages or summaries retrieved from memory.

    Returns:
        `None`
    """
    # Get current timestamp
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Set random seed for reproducibility
    random.seed(random_seed)
    
    # Set openai api key
    openai.api_key = open_ai_key

    # Define model configs and settings
    model_configs = [
        {
            "model_type": "openai_chat",
            "config_name": fast_mind_model,
            "model_name": fast_mind_model,
            "api_key": openai_key,
            "generate_args": {
                "temperature": 1,
            }
        },
        {
            "model_type": "openai_chat",
            "config_name": slow_mind_model,
            "model_name": slow_mind_model,
            "api_key": openai_key,
            "generate_args": {
                "temperature": 1,
            }
        }
    ]


    for game_num in range(0, num_games):
        # TSV log file params
        game_tsv_file = f"{basefilepath}/{'reflect' if reflect_before_vectorstore else 'noreflect'}/{random_seed}/{timestamp_str}/fastslow_raw_game{game_num}.tsv"
        os.makedirs(os.path.dirname(game_tsv_file), exist_ok=True)
        tsv_logger = StructuredLogger(game_tsv_file)
        
        config = {
                "max_days_per_game": max_days_per_game,
                "num_games": num_games,
                "reflect_before_vectorstore": reflect_before_vectorstore,
                "fast_mind_model": fast_mind_model,
                "slow_mind_model": slow_mind_model,
                "max_werewolf_discussion_round": max_werewolf_discussion_round, 
                "similarity_top_k": similarity_top_k
        }
        tsv_logger.log("game_init", config)
        
        # Define the config settings for each agent involved
        agent_configs = [
            {
                "class": "SlowFastMindAgent",
                "args": {
                    "name": "Player1",
                    "sys_prompt": f"""
                        You are a strategic decision-maker playing a game of werewolf.

                        You are Player1 and there are totally 4 players, named Player1, Player2, Player3, Player4.

                        Rules of the game:

                        {Prompts.rules}

                        You are playing werewolf in this game.
                    """,
                    "fast_mind_model_config_name": fast_mind_model,
                    "slow_mind_model_config_name": slow_mind_model,
                    "reflect_before_vectorstore": reflect_before_vectorstore,
                    "similarity_top_k": similarity_top_k,
                    "openai_api_key": open_ai_key,
                    "tsv_logger": tsv_logger
                }
            },
            {
                "class": "NormalAgent",
                "args": {
                    "name": "Player2",
                    "sys_prompt": f""" 
                        You are a strategic decision-maker playing a game of werewolf.

                        You are Player2 and there are totally 4 players, named Player1, Player2, Player3, Player4.

                        Rules of the game: 

                        {Prompts.rules}

                        You are playing villager in this game.
                    """,
                    "model_config_name": fast_mind_model,
                    "similarity_top_k": similarity_top_k,
                    "openai_api_key": open_ai_key,
                    "tsv_logger": tsv_logger
                }
            },
            {
                "class": "NormalAgent",
                "args": {
                    "name": "Player3",
                    "sys_prompt": f""" 
                        You are a strategic decision-maker playing a game of werewolf.

                        You are Player3 and there are totally 4 players, named Player1, Player2, Player3, Player4.

                        Rules of the game:

                        {Prompts.rules}

                        You are playing seer in this game.
                    """,
                    "model_config_name": fast_mind_model,
                    "similarity_top_k": similarity_top_k,
                    "openai_api_key": open_ai_key,
                    "tsv_logger": tsv_logger
                }
            },
            {
                "class": "NormalAgent",
                "args": {
                    "name": "Player4",
                    "sys_prompt": f""" 
                        You are a strategic decision-maker playing a game of werewolf.

                        You are Player4 and there are totally 4 players, named Player1, Player2, Player3, Player4.

                        Rules of the game:

                        {Prompts.rules}

                        You are playing witch in this game.
                    """,
                    "model_config_name": fast_mind_model,
                    "similarity_top_k": 1,
                    "openai_api_key": open_ai_key,
                    "tsv_logger": tsv_logger
                }
            }
        ]
        
        # Read model and agent configs, and initialize agents automatically
        survivors = agentscope.init(
            model_configs=model_configs,
            agent_configs=agent_configs,
            project="Werewolf"
        )

        # Get player roles
        roles = ["werewolf", "villager", "seer", "witch"]
        wolves, witch, seer, villagers = [survivors[0]], survivors[-1], survivors[-2], [survivors[-3]]

        # Initialize game state
        game_state = {
            "werewolves": [player.name for player in wolves],
            "villagers": [player.name for player in survivors if player not in wolves],
            "seer": [seer.name],
            "witch": [witch.name],
            "survivors": [player.name for player in survivors],
            "dead": [],
            "witch_healing_used": False,
            "witch_poison_used": False,
            "endgame": False,
            "winner": None
        }

        current_log_path = generate_log_filepath(basefilepath, reflect_before_vectorstore, random_seed, timestamp_str, game_num)
        change_log_file(current_log_path)

        # Log parameters
        log_params(
            current_log_path, 
            basefilepath=basefilepath, 
            max_days_per_game=max_days_per_game, 
            num_games=num_games,
            reflect_before_vectorstore=reflect_before_vectorstore, 
            max_werewolf_discussion_round=max_werewolf_discussion_round, 
            similarity_top_k=similarity_top_k,
            random_seed=random_seed
        )

        logger.info(f"Starting game {game_num}")

        custom_agent_won = run_game(
            slow_mind_model,
            max_days_per_game,
            reflect_before_vectorstore,
            max_werewolf_discussion_round,
            wolves,
            seer,
            witch,
            roles,
            survivors,
            game_state,
            tsv_logger
        )

        wolves[0].save_context(timestamp_str, game_num)
        witch.save_context(timestamp_str, game_num, "witch")
        seer.save_context(timestamp_str, game_num, "seer")
        villagers[0].save_context(timestamp_str, game_num, "villager")

        row = {
            "raw_log_filepath": current_log_path,
            "custom_agent_won": custom_agent_won
        }
        save_results(f"{basefilepath}/{'reflect' if reflect_before_vectorstore else 'noreflect'}/{random_seed}/{timestamp_str}/fastslow_win_rate.csv", row)
        

In [13]:
run_experiment(
    open_ai_key=openai_key,
    fast_mind_model = "gpt-4o-mini",
    slow_mind_model = "gpt-4o",
    basefilepath="logs/fastslow",
    reflect_before_vectorstore=False,
    max_days_per_game=2,
    num_games=2,
    similarity_top_k=2,
    random_seed=43
)

2025-02-27 14:17:51 | INFO     | agentscope.manager._model:load_model_configs:115 - Load configs for model wrapper: gpt-4o-mini, gpt-4o
2025-02-27 14:17:51 | INFO     | agentscope.models.model:__init__:203 - Initialize model by configuration [gpt-4o-mini]
2025-02-27 14:17:51 | WARNING  | agentscope.models.openai_model:__init__:117 - fail to get max_length for gpt-4o-mini: 'Model [gpt-4o-mini] not found in OPENAI_MAX_LENGTH. The last updated date is 20231212'
2025-02-27 14:17:51 | INFO     | agentscope.models.model:__init__:203 - Initialize model by configuration [gpt-4o]
2025-02-27 14:17:51 | WARNING  | agentscope.models.openai_model:__init__:117 - fail to get max_length for gpt-4o: 'Model [gpt-4o] not found in OPENAI_MAX_LENGTH. The last updated date is 20231212'
2025-02-27 14:17:51 | INFO     | agentscope.models.model:__init__:203 - Initialize model by configuration [gpt-4o-mini]
2025-02-27 14:17:51 | WARNING  | agentscope.models.openai_model:__init__:117 - fail to get max_length for

2025-02-27 14:17:51 - INFO - Starting game 0
2025-02-27 14:17:51 - INFO - Moderator: Player1, if you are the only werewolf, eliminate a player. Otherwise, discuss with your teammates and reach an agreement.
2025-02-27 14:17:51 - INFO - Moderator: Player1, if you are the only werewolf, eliminate a player. Otherwise, discuss with your teammates and reach an agreement.
2025-02-27 14:17:51 - INFO - ===
2025-02-27 14:17:51 - INFO - Query: Discussions related to villagers, seer, witch.
2025-02-27 14:17:51 - INFO - Retrieved similar documents: 


Parameters written to logs/fastslow/noreflect/43/20250227_141751/fastslow_raw_game0.log
Moderator: Player1, if you are the only werewolf, eliminate a player. Otherwise, discuss with your teammates and reach an agreement.


2025-02-27 14:17:51 - INFO - ===
2025-02-27 14:17:53 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Player1: ```json
{"thought": "I need to carefully observe how the other players are acting. I suspect Player2 might be a werewolf due to their inconsistent statements about previous night events. I should bring this up to gain support for my accusation.", "speak": "I've been noticing some inconsistencies in what Player2 mentioned regarding last night’s events. It’s important we discuss this carefully since we’re down to just four players. What does everyone else think?", "finish_discussion": "false"}
```


...

Player3: ```json
{"thought": "I need to analyze the discussions and behavior of the players to deduce who might be a werewolf. After considering the clues, my suspicion points towards Player2 as they seem to be deflecting blame.", "vote": "Player2", "uncertainty_score": "0.3"}
```


2025-02-27 14:18:51 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-27 14:18:52 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-27 14:18:52 - INFO - ===
2025-02-27 14:18:52 - INFO - Query: Discussions related to werewolves.
2025-02-27 14:18:52 - INFO - Retrieved similar documents: Player4: {"thought": "Eliminating Player3 is a key decision. I need to consider if they were a threat or if there's potential value in keeping them alive. If I'm suspicious of Player3 being a werewolf, I might not choose to resurrect them, but if they could provide valuable information, it might be worth it.", "speak": "I'm going to resurrect Player3 because I believe they can provide valuable insights into who the werewolves might be, and we need their information to make a better decision moving forward.", "resurrect": true}
2025-02-27 14:18:52 - INFO - ===
2025-02-27 14:18:53 - INFO - HTTP Request: POST https://api.openai

Player4: ```json
{"thought": "I suspect Player1 is a werewolf based on their behavior during discussions. They seem to be deflecting attention away from themselves. I feel confident about this assessment.", "vote": "Player1", "uncertainty_score": 0.2}
```


2025-02-27 14:18:53 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-27 14:18:53 - INFO - Moderator: Player1 has been voted out.
2025-02-27 14:18:53 - INFO - Moderator: The game is over. The werewolves have been defeated, and the village is safe once again!
2025-02-27 14:18:53 - INFO - Moderator: The game is over. The werewolves have been defeated, and the village is safe once again!


Moderator: Player1 has been voted out.
Messages saved to vectorstore\fastslow\20250227_141751\1\werewolf.log
Messages saved to vectorstore\fastslow\20250227_141751\1\witch.log
Messages saved to vectorstore\fastslow\20250227_141751\1\seer.log
Messages saved to vectorstore\fastslow\20250227_141751\1\villager.log
Row added to logs/fastslow/noreflect/43/20250227_141751/fastslow_win_rate.csv: {'raw_log_filepath': 'logs/fastslow/noreflect/43/20250227_141751/fastslow_raw_game1.log', 'custom_agent_won': False}
